## fe_and_lgb2 파일에서 50개 주요 변수만 추출하여 결과 비교해 보기
[Feature Engineering & LightGBM](https://www.kaggle.com/davidcairuz/feature-engineering-lightgbm)
- 정확도: 0.9300
- 근데 이 파라미터가 50개만 남긴 데이터에 맞는 파라미터일까. 파라미터 추출하는 방법은 뭐지?? 베이지안?
- 이제 email_binary 등 해서 결과 비교해 보기
- [디스커션](https://www.kaggle.com/c/ieee-fraud-detection/discussion/107697#latest-622651) -> 강한 상관관계 있는 열들 제거해 보기. 그럼 모델이 조금 더 중요한 변수를 학습하는 데 집중할 수 있을 것
- pd.rolling -> Moving Average. 이거 고려해 보기. noise 때문에 효과가 떨어진 거라면, 스코어 좋아질 수 있음

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from sklearn.metrics import roc_auc_score
import gc, os, pickle

from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

for dirname, _, filenames in os.walk("../input/"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../input/df_id.pkl
../input/df_pca.pkl
../input/df_test.pkl
../input/df_train.pkl
../input/df_trans.pkl
../input/sample_submission.csv
../input/second_test.pkl
../input/second_train.pkl
../input/test.pkl
../input/test_0823.pkl
../input/test_identity.csv
../input/test_transaction.csv
../input/train_0823.pkl
../input/train_identity.csv
../input/train_transaction.csv


In [34]:
%%time
train = pd.read_pickle("../input/second_train.pkl")
test = pd.read_pickle("../input/second_test.pkl")

Wall time: 1.52 s


In [35]:
# 칼럼 수 확인 -> drop 안한 것(488개)
print(train.shape)
print(test.shape)

(590540, 488)
(506691, 487)


In [36]:
# 이 데이터는 TransactionDT 파싱 안한 거였네. 하기!!
import datetime
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')

for df in [train, test]:
    # Temporary
    df['DT'] = df['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
    df['DT_M'] = (df['DT'].dt.year-2017)*12 + df['DT'].dt.month
    df['DT_W'] = (df['DT'].dt.year-2017)*52 + df['DT'].dt.weekofyear
    df['DT_D'] = (df['DT'].dt.year-2017)*365 + df['DT'].dt.dayofyear
    
    df['DT_hour'] = df['DT'].dt.hour
    df['DT_day_week'] = df['DT'].dt.dayofweek
    df['DT_day'] = df['DT'].dt.day
    
    # D9 column. 이건 왜 하는지 모르겠지만 일단 해 놓기
    df['D9'] = np.where(df['D9'].isna(),0,1)

In [18]:
with open('top5.pkl', 'rb') as f:
    top50_features = pickle.load(f)

In [19]:
top50_features.head()

,feature,fold_1,fold_2,fold_3,fold_4,fold_5,average
471,card1_count_full,32421,28017,24038,23633,47597,31141.2
488,DT_D,33006,27305,24447,23921,46181,30972.0
470,addr1__card1,32355,27232,24316,24293,46044,30848.0
2,card1,32308,27460,22909,22645,42433,29551.0
467,card1__card5,29188,25945,22599,22023,41829,28316.8


In [37]:
train = train.sort_values('TransactionDT')[list(['isFraud']) + list(top50_features['feature'])]
test = test.sort_values('TransactionDT')[top50_features['feature']]

In [38]:
# 칼럼 수 확인
print(train.shape)
print(test.shape)

(590540, 51)
(506691, 50)


In [39]:
train.columns

Index(['isFraud', 'card1_count_full', 'DT_D', 'addr1__card1', 'card1',
       'card1__card5', 'TransactionAmt_to_std_card1', 'card2',
       'card2_count_full', 'TransactionAmt_to_mean_card1', 'addr1', 'DT_day',
       'card2__dist1', 'TransactionAmt_to_std_addr1',
       'TransactionAmt_to_mean_addr1', 'TransactionAmt_to_mean_card4',
       'card5__P_emaildomain', 'TransactionAmt_to_std_card4',
       'P_emaildomain__C2', 'Transaction_hour', 'DT_W', 'dist1',
       'TransactionAmt', 'card2__id_20', 'D4', 'D2', 'D15_to_mean_card1',
       'TransactionAmt_decimal', 'D15_to_std_card1', 'D10', 'DT_hour', 'C13',
       'D1', 'DeviceInfo__P_emaildomain', 'Transaction_day_of_week', 'D11',
       'id_20', 'D15_to_mean_addr1', 'DT_day_week', 'id_19',
       'D15_to_mean_card4', 'card5', 'C1', 'D8', 'P_emaildomain', 'D15',
       'D15_to_std_addr1', 'card5_count_full', 'C2', 'D15_to_std_card4',
       'P_emaildomain_bin'],
      dtype='object')

In [40]:
from sklearn.model_selection import KFold
import lightgbm as lgb

params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47,
         }

In [41]:
%%time
# TransactionDT, DT 칼럼은 이미 50개 컬럼 추출할 때 제거됨
X = train.drop(['isFraud'], axis=1)
y = train['isFraud']

X_test = test

del train, test
gc.collect()

Wall time: 553 ms


4867

In [42]:
%%time
# Nfold 원래 5였는데, 6으로 해 보기!!
NFOLDS = 6
folds = KFold(n_splits=NFOLDS)

columns = X.columns
splits = folds.split(X, y)
y_preds = np.zeros(X_test.shape[0])
y_oof = np.zeros(X.shape[0])
score = 0

feature_importances = pd.DataFrame()
feature_importances['feature'] = columns
  
for fold_n, (train_index, valid_index) in enumerate(splits):
    X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_valid, label=y_valid)

    clf = lgb.train(params, dtrain, 10000, valid_sets = [dtrain, dvalid], verbose_eval=200, early_stopping_rounds=500)
    
    feature_importances[f'fold_{fold_n + 1}'] = clf.feature_importance()
    
    y_pred_valid = clf.predict(X_valid)
    y_oof[valid_index] = y_pred_valid
    print(f"Fold {fold_n + 1} | AUC: {roc_auc_score(y_valid, y_pred_valid)}")
    
    score += roc_auc_score(y_valid, y_pred_valid) / NFOLDS
    y_preds += clf.predict(X_test) / NFOLDS
    
    del X_train, X_valid, y_train, y_valid
    gc.collect()
    
print(f"\nMean AUC = {score}")
print(f"Out of folds AUC = {roc_auc_score(y, y_oof)}")

Training until validation scores don't improve for 500 rounds.
[200]	training's auc: 0.962168	valid_1's auc: 0.883048
[400]	training's auc: 0.978531	valid_1's auc: 0.895265
[600]	training's auc: 0.987785	valid_1's auc: 0.901008
[800]	training's auc: 0.993352	valid_1's auc: 0.903592
[1000]	training's auc: 0.996523	valid_1's auc: 0.905508
[1200]	training's auc: 0.998256	valid_1's auc: 0.907046
[1400]	training's auc: 0.999109	valid_1's auc: 0.907469
[1600]	training's auc: 0.999544	valid_1's auc: 0.908148
[1800]	training's auc: 0.999769	valid_1's auc: 0.908125
[2000]	training's auc: 0.999887	valid_1's auc: 0.908501
[2200]	training's auc: 0.999946	valid_1's auc: 0.908699
[2400]	training's auc: 0.999974	valid_1's auc: 0.908534
[2600]	training's auc: 0.999987	valid_1's auc: 0.90835
Early stopping, best iteration is:
[2204]	training's auc: 0.999947	valid_1's auc: 0.908702
Fold 1 | AUC: 0.9087083826259736
Training until validation scores don't improve for 500 rounds.
[200]	training's auc: 0.962

In [43]:
# 1600 사이클까지 봤는데 val auc가 잘 안오르네.. 기대는 안됨
sub = pd.read_csv('../input/sample_submission.csv')
sub['isFraud'] = y_preds
sub.to_csv("0916_top50_submission.csv", index=False)